## Problem 1: Reinforcement Learning: Policy Iteration (10 Points)

1. Suppose we have a policy $\pi$ and a monotonically improved policy $\pi'$, such that for all states $s \in S$, we have 

$$
Q_{\pi}(s, \pi'(s)) \ge V_{\pi}(s).
$$

Show that the value function $V_{\pi'}$ dominates $V_{\pi}$, i.e., for all states $s \in S$

$$
V_{\pi'}(s) \ge V_{\pi}(s).
$$

*hint: Consider expanding the value functions iteratively.*

2. Apply the conclusion of part 1 to the policy improvement step to show that it leads to a sequence of monotonically improving policies. In other words, show that if $\pi \neq \pi^*$, the next round policy $\pi'$ under the policy iteration algorithm satisfies $V_{\pi'}(s) > V_{\pi}(s)$ for some state $s\in S$.

![](https://rstr.in/avjn9edat3zttr/my-library/DpCIEHmbGUj)

worked with valentina

## Problem 2: Elephants Can Remember (20 points)

In [3]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/85/15/cf99a373812d37f8ae99752a34a9f5f690d820ceb5b302e922705bc18944/tensorflow-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for tensorflow-macos==2.15.0 from https://files.pythonhosted.org/packages/eb/9f/0759e2fea4a3c48f070b64811c2c57036b46353ba87263afc810b8f4188a/tensorflow_macos-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-non

In [4]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import random
  
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation
from keras.layers import GRU, SimpleRNN

In [6]:
with open('q2_data/Agatha_Christie_train.txt', 'r') as file:
    train_text = file.read()
    
with open('q2_data/Agatha_Christie_test.txt', 'r') as file:
    test_text = file.read()

vocabulary = sorted(list(set(train_text + test_text)))
vocab_size = len(vocabulary)

# Dictionaries to go from a character to index and vice versa
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

In [7]:
np.random.choice?

In [8]:
# The first 500 characters of our training set
train_text[0:500]

'mrs. oliver looked at herself in the glass. she gave a brief, sideways look towards the clock on the mantelpiece, which she had some idea was twenty minutes slow. then she resumed her study of her coiffure. the trouble with mrs. oliver was--and she admitted it freely--that her styles of hairdressing were always being changed. she had tried almost everything in turn. a severe pompadour at one time, then a wind-swept style where you brushed back your locks to display an intellectual brow, at least'

In [9]:
print("The vocabulary contains", vocab_size, "characters")
print("The training set contains", len(train_text) ,"characters")
print("The test set contains", len(test_text) ,"characters")

The vocabulary contains 44 characters
The training set contains 262174 characters
The test set contains 7209 characters


### Problem 2.1: The Diversity of Language Models

#### Vanilla RNNs

Simplicity: Vanilla RNNs have a relatively simple structure. They consist of a single hidden layer with a recurrent connection that feeds the hidden state from one step of the sequence to the next.

Issue with Long-Term Dependencies: Vanilla RNNs struggle with capturing long-term dependencies in a sequence. This is primarily due to the vanishing gradient problem, where gradients become exponentially smaller as they are propagated back through each time step. As a result, it becomes difficult for the RNN to learn and maintain information from earlier time steps in longer sequences.

Training Difficulty: Due to the vanishing gradient problem, training vanilla RNNs on long sequences is challenging, as the network becomes less sensitive to input and changes in parameters for early elements in the sequence.

#### GRUs

Complex Architecture with Gates: GRUs are an advancement over vanilla RNNs. They incorporate gating mechanisms - specifically, the update gate and reset gate - which control the flow of information.
Update Gate: The update gate in a GRU helps the model determine how much of the past information (from previous time steps) needs to be passed along to the future. This is crucial for the model to 'remember' relevant information over long sequences.

Reset Gate: The reset gate decides how much past information to forget. This is useful when the sequence has segments with little or no relevance to each other, allowing the model to drop irrelevant information from the past.

Addressing Vanishing Gradient Problem: The gating mechanisms in GRUs help mitigate the vanishing gradient problem seen in vanilla RNNs. By adaptively choosing which information to pass through, GRUs can maintain relevant information over long sequences, making them more effective for tasks involving long-term dependencies.

Efficiency in Learning Long Sequences: Due to their gating mechanisms, GRUs are generally more efficient and perform better than vanilla RNNs in learning from long sequences. They are capable of capturing dependencies from a larger range of time steps.

### Problem 2.2: Generating Text with the Vanilla RNN

1. Character-based models are not limited by a fixed vocabulary, unlike word-based models which can struggle with words not seen during training. In word-based models, any word not in the training vocabulary is typically treated as an 'unknown' or OOV token, which limits the model's ability to understand or generate those words. Character-based models, on the other hand, can handle any word, even if it was not seen during training, as long as the characters making up the word are known.

2. Character-based models can learn and generate new, plausible word forms that may not have been present in the training data. This is because they model the language at a more granular level, capturing underlying patterns in how characters combine to form words. This could be particularly useful in applications like poetry, rhyme, or prose generation, or in languages where compound words are common.


In [10]:
# load json and create model
json_file = open('q2_data/RNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

RNN_model = model_from_json(loaded_model_json)
RNN_model.load_weights("q2_data/RNN_model.h5")

In [11]:
# load in the weights and show summary
weights_RNN = RNN_model.get_weights()
RNN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Vanilla_RNN_1 (SimpleRNN)   (None, 100, 128)          22144     
                                                                 
 Vanilla_RNN_2 (SimpleRNN)   (None, 64)                12352     
                                                                 
 Dense_layer (Dense)         (None, 44)                2860      
                                                                 
 Softmax_layer (Activation)  (None, 44)                0         
                                                                 
Total params: 37356 (145.92 KB)
Trainable params: 37356 (145.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
def sample_text_RNN(weights, prompt, N):
    '''
    Uses a pretrained RNN to generate text, starting from a prompt, 
    only using the weights and numpy commands
            Parameters:
                    weights (list): Weights of the pretrained RNN model
                    prompt (string): Start of generated sentence
                    N (int): Length of output sentence (including prompt)
            Returns:
                    output_sentence (string): Text generated by RNN
    '''
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides

    # First Recurrent Layer 
    W_xh1 = weights[0].T 
    W_h1h1 = weights[1].T 
    b_h1 = np.expand_dims(weights[2], axis=1)

    # Second Recurrent Layer
    W_h1h2 = weights[3].T
    W_h2h2 = weights[4].T
    b_h2 = np.expand_dims(weights[5], axis=1)

    # Linear (dense) layer
    W_h2y = weights[6].T
    b_y = np.expand_dims(weights[7], axis=1)
    
    # Initiate the hidden states
    h1 = np.zeros((W_h1h1.shape[0], 1))
    h2 = np.zeros((W_h2h2.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    output_sentence = prompt
    
    # Function to convert character to one-hot vector
    def char_to_onehot(char):
        onehot = np.zeros((vocab_size, 1))
        onehot[char_to_indices[char]] = 1
        return onehot

    # Function for softmax
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    for i in range(N - len(prompt)):
        # Get the input character
        x = char_to_onehot(output_sentence[-1]) if i > 0 else char_to_onehot(prompt[0])

        # First Recurrent Layer
        h1 = np.tanh(np.dot(W_xh1, x) + np.dot(W_h1h1, h1) + b_h1)

        # Second Recurrent Layer
        h2 = np.tanh(np.dot(W_h1h2, h1) + np.dot(W_h2h2, h2) + b_h2)

        # Output Layer
        y = np.dot(W_h2y, h2) + b_y
        p = softmax(y)

        # Sample a character from the probability distribution
        idx = np.random.choice(range(vocab_size), p=p.ravel())
        output_sentence += indices_to_char[idx]

    return output_sentence

In [43]:
print(sample_text_RNN(weights_RNN, 
                      'mrs. oliver looked at herself in the glass. she gave a brief, sideways look', 
                      1000))

mrs. oliver looked at herself in the glass. she gave a brief, sideways lookand or somother who had beendrements. she mateesthard the ladd the was a maid, i bely as orcoughtous to askitt you going coxtaly. theing of serso." "yes, no no. not shive? she suplofe muringany time invily yellntale are at wantiduse it sascilious and thought there was anly a rowadd foor and they asor for and sormon. alint: said then i such whom a surt anyowy pose anyon,." "yes, she cailst, of cor's whow you said pave for a mrs. oliver baskby mrs. oliver. oh, you?" "yes, someone shameectul, is as such other there were he fion in aly will be wis simely wills quicadred a frouk ham mantersulf oher the fame nations were expmened her mothers and triep and explened lithoride. ane, i've you telnought exteclearint it any oldoy in the cheridencably to anywhat anything. nimily, he was madnere noo." "they raglly wyone wan the came you loubor, of celia realy," said poirot. camirat of ollcagre." "and you mentalowe mightold an

### Problem 2.3: Generating Text with the GRU

In [15]:
# load json and create model
json_file = open('q2_data/GRU_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

GRU_model = model_from_json(loaded_model_json)
GRU_model.load_weights("q2_data/GRU_model.h5")

In [16]:
# load in the weights and show summary
weights_GRU = GRU_model.get_weights()
GRU_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 512)               857088    
                                                                 
 dense (Dense)               (None, 44)                22572     
                                                                 
 activation (Activation)     (None, 44)                0         
                                                                 
Total params: 879660 (3.36 MB)
Trainable params: 879660 (3.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# Helper function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sample_text_GRU(weights, prompt, N):
    '''
    Uses a pretrained GRU to generate text, starting from a prompt,
    only using the weights and numpy commands
            Parameters:
                    weights (list): Weights of the pretrained GRU model
                    prompt (string): Start of generated sentence
                    N (int): Total length of output sentence
            Returns:
                    output_sentence (string): Text generated by GRU
    '''
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides
    
    # GRU Layer 
    W_ux, W_rx, W_hx = np.split(weights[0].T, 3, axis = 0)
    W_uh, W_rh, W_hh = np.split(weights[1].T, 3, axis = 0)

    bias = np.sum(weights[2], axis=0)
    b_u, b_r, b_h = np.split(np.expand_dims(bias, axis=1), 3)

    # Linear (dense) layer
    W_y = weights[3].T
    b_y = np.expand_dims(weights[4], axis=1)
    
    # Initiate hidden state
    h = np.zeros((W_hh.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    output_sentence = ""
        
     # Helper function for softmax
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    # Function to convert character to one-hot vector
    def char_to_onehot(char):
        onehot = np.zeros((vocab_size, 1))
        onehot[char_to_indices[char]] = 1
        return onehot

    output_sentence = prompt

    for i in range(N - len(prompt)):
        # Get the input character
        x = char_to_onehot(output_sentence[-1]) if i > 0 else char_to_onehot(prompt[0])

        # GRU Forward Pass
        u = sigmoid(np.dot(W_ux, x) + np.dot(W_uh, h) + b_u)
        r = sigmoid(np.dot(W_rx, x) + np.dot(W_rh, h) + b_r)
        h_tilde = np.tanh(np.dot(W_hx, x) + r * (np.dot(W_hh, h)) + b_h)
        h = u * h + (1 - u) * h_tilde

        # Output Layer
        y = np.dot(W_y, h) + b_y
        p = softmax(y)

        # Sample a character from the probability distribution
        idx = np.random.choice(range(vocab_size), p=p.ravel())
        output_sentence += indices_to_char[idx]

    return output_sentence

In [53]:
print(sample_text_GRU(weights_GRU, 
                      'mrs. oliver looked at herself in the glass. she gave a brief, sideways look',
                      1000))

mrs. oliver looked at herself in the glass. she gave a brief, sideways looky to her so readly the dobto that. i to going the naverally deat on the finger to it. in to call the ation at the time or youred the warkerite of sef she was at eeepll showing will hive he will call living the aidepoilot the donot along goron ano souts of celie had interestion putternt and whe that she had a get an son a sthacis and the wefe. i mean the stway was he wanted to lear that see worried. it's a troubligs this suseere aill dis in the went all about. the ho tooull you ask a young moment or truat from oriones ard elephants and i had mide the dount gitl dearing in erea list, some ixa ten uple tig things they were not snowe him. but there ask there is as it listed telbe how inferettence. and ame. at the eamy a pimp rime mo gaing i a to tay it, but all ary old quite a tranked at about it or me the rather nive. i seetlight tells that's the other recently dades, a telline, because it they we loor that. some h

### Vanilla RNN Output: 

This output exhibited less coherence in terms of sentence structure and logical progression of ideas. Vanilla RNNs can struggle with long-term dependencies due to issues like vanishing gradients, making it harder for them to maintain context over longer sequences. There are more repetitive patterns and broken sentence structures, as the model may lose track of what it has already generated. Here we see it struggles more with maintaining consistent use of named entities. Characters and subjects seem to appear and change abruptly without clear context. The text also exhibits a certain level of repetitiveness and abrupt topic shifts ("she cailst, of cor's whow you said pave for a mrs. oliver baskby mrs. oliver"). The output displays a somewhat disjointed and erratic flow of ideas. The sentences are more fragmented, and there are noticeable grammatical inconsistencies and nonsensical phrases ("mateesthard the ladd the was a maid", "yellntale are at wantiduse"). It suggests a struggle to maintain context and a coherent narrative over the sequence. Clearly higher perplexity.


### GRU Output: 

This output generated far less gibberish and seemed to have more contextually relevant words over various iterations. This is likely because GRUs, with their gating mechanisms are generally better at capturing long-term dependencies in sequences. The text shows a somewhat better grasp of character and subject continuity, although it's not perfect. While there is some repetition, it's less pronounced. The GRU model shows a better ability to vary sentence structure and content, which can be attributed to its more sophisticated handling of sequence dependencies.The text appears more coherent and maintains a clearer narrative structure. While there are still some grammatical errors and odd phrases, the sentences are more structured and logical ("may she things with some interesten to me semition and the sime by scaroied, but whem canell from i who leove now she looked at you?" "oh, yes, she looked a oni?. i shill remember to know any owe now fir this. i suppose i well to me?"). This form of text has a much higher occurance of sensible words and sentence structure. This indicates a better handling of context and language rules. Clearly lower perplexity

2. Calculate the perplexity of each language model by using test_text, an unseen extract of the book. Choose the prompt as the first $m$ letters of the test set, where $m$ is a parameter that you can choose yourself. You should be able to reuse the majority of your previous code in this calculation. Discuss your results at the end.

In [62]:
def sample_text_RNN(weights, prompt, N):
    '''
    Uses a pretrained RNN to generate text, starting from a prompt, 
    only using the weights and numpy commands
            Parameters:
                    weights (list): Weights of the pretrained RNN model
                    prompt (string): Start of generated sentence
                    N (int): Length of output sentence (including prompt)
            Returns:
                    output_sentence (string): Text generated by RNN
    '''
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides

    # First Recurrent Layer 
    W_xh1 = weights[0].T 
    W_h1h1 = weights[1].T 
    b_h1 = np.expand_dims(weights[2], axis=1)

    # Second Recurrent Layer
    W_h1h2 = weights[3].T
    W_h2h2 = weights[4].T
    b_h2 = np.expand_dims(weights[5], axis=1)

    # Linear (dense) layer
    W_h2y = weights[6].T
    b_y = np.expand_dims(weights[7], axis=1)
    
    # Initiate the hidden states
    h1 = np.zeros((W_h1h1.shape[0], 1))
    h2 = np.zeros((W_h2h2.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    output_sentence = prompt
    sum_log_probs = 0 
    
    def char_to_onehot(char):
        onehot = np.zeros((vocab_size, 1))
        onehot[char_to_indices[char]] = 1
        return onehot

    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    for i in range(N - len(prompt)):
        # Get the input character
        x = char_to_onehot(output_sentence[-1]) if i > 0 else char_to_onehot(prompt[0])

        # First Recurrent Layer
        h1 = np.tanh(np.dot(W_xh1, x) + np.dot(W_h1h1, h1) + b_h1)

        # Second Recurrent Layer
        h2 = np.tanh(np.dot(W_h1h2, h1) + np.dot(W_h2h2, h2) + b_h2)

        # Output Layer
        y = np.dot(W_h2y, h2) + b_y
        p = softmax(y)

        # calculate log probability of actual next character
        if i < len(test_text) - len(prompt):
            actual_next_char = test_text[len(prompt) + i]
            actual_next_char_idx = char_to_indices[actual_next_char]
            prob = p[actual_next_char_idx]
            sum_log_probs += np.log(prob + 1e-9)  # avoid log(0)
        
        # Sample a character from the probability distribution
        idx = np.random.choice(range(vocab_size), p=p.ravel())
        output_sentence += indices_to_char[idx]

    return output_sentence, sum_log_probs

In [65]:
# Helper function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sample_text_GRU(weights, prompt, N):
    '''
    Uses a pretrained GRU to generate text, starting from a prompt,
    only using the weights and numpy commands
            Parameters:
                    weights (list): Weights of the pretrained GRU model
                    prompt (string): Start of generated sentence
                    N (int): Total length of output sentence
            Returns:
                    output_sentence (string): Text generated by GRU
    '''
    # Extracting weights and biases
    # Dimensions of matrices are same format as lecture slides
    
    # GRU Layer 
    W_ux, W_rx, W_hx = np.split(weights[0].T, 3, axis = 0)
    W_uh, W_rh, W_hh = np.split(weights[1].T, 3, axis = 0)

    bias = np.sum(weights[2], axis=0)
    b_u, b_r, b_h = np.split(np.expand_dims(bias, axis=1), 3)

    # Linear (dense) layer
    W_y = weights[3].T
    b_y = np.expand_dims(weights[4], axis=1)
    
    # Initiate hidden state
    h = np.zeros((W_hh.shape[0], 1))
    
    # -----------------------------------------------
    
    # Your code starts here
    output_sentence = ""
        
     # Helper function for softmax
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    # Function to convert character to one-hot vector
    def char_to_onehot(char):
        onehot = np.zeros((vocab_size, 1))
        onehot[char_to_indices[char]] = 1
        return onehot

    output_sentence = prompt
    sum_log_probs = 0

    for i in range(N - len(prompt)):
        # Get the input character
        x = char_to_onehot(output_sentence[-1]) if i > 0 else char_to_onehot(prompt[0])

        # GRU Forward Pass
        u = sigmoid(np.dot(W_ux, x) + np.dot(W_uh, h) + b_u)
        r = sigmoid(np.dot(W_rx, x) + np.dot(W_rh, h) + b_r)
        h_tilde = np.tanh(np.dot(W_hx, x) + r * (np.dot(W_hh, h)) + b_h)
        h = u * h + (1 - u) * h_tilde

        # Output Layer
        y = np.dot(W_y, h) + b_y
        p = softmax(y)
        
        # calculate log probability of actual next character
        if i < len(test_text) - len(prompt):
            actual_next_char = test_text[len(prompt) + i]
            actual_next_char_idx = char_to_indices.get(actual_next_char, None)
            if actual_next_char_idx is not None:
                prob = p[actual_next_char_idx]
                sum_log_probs += np.log(prob + 1e-9)  # avoid log(0)


        # Sample a character from the probability distribution
        idx = np.random.choice(range(vocab_size), p=p.ravel())
        output_sentence += indices_to_char[idx]
        

    return output_sentence, sum_log_probs

In [66]:
def calculate_perplexity(model_function, weights, prompt, test_sequence):
    # Generate text using the model
    generated_text, log_probs = model_function(weights, prompt, len(test_sequence))
    
    # Calculate perplexity
    perplexity = np.exp(-np.sum(log_probs) / len(test_sequence))
    return perplexity

# Choose a prompt length
m = 50  # For example, the first 50 characters
prompt = test_text[:m]

# Compute perplexity for both models
perplexity_RNN = calculate_perplexity(sample_text_RNN, weights_RNN, prompt, test_text)
perplexity_GRU = calculate_perplexity(sample_text_GRU, weights_GRU, prompt, test_text)

# Print results
print(f"Perplexity of Vanilla RNN: {perplexity_RNN}")
print(f"Perplexity of GRU: {perplexity_GRU}")


Perplexity of Vanilla RNN: 1126.5341368285174
Perplexity of GRU: 540.5134506477787


#### As discussed before, GRU expectedly has lower perplexity, indicating a better model. This is expected

Clearly, we saw that the GRU approach is much much better. However we can improve upon this...

Implementing a bidirectional rather than unidirectional feed-forward GRU as implemented could be beneficial. The most significant advantage of a bidirectional GRU is its ability to process the input sequence from both forward and backward directions. This means the model would capture information from both past and future contexts relative to a given point in the sequence, leading to better entity recognition, sentiment analysis, and part-of-speech tagging. The context available on both sides of a given word or token can be crucial for accurate predictions.

In addition, implementing attention mechanisms could help the model better determine the relevant parts of input and develop a more contextually appropriate response